In [1]:
!pip install --user uproot
!pip install --user tqdm
!pip install awkward
!pip install xgboost
!pip install sklearn

In [1]:
import uproot
import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
import awkward as ak
import ROOT
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.24/02


In [2]:
root_files=['WWG_emu','ST','TTGJets','ZGJets','WGJets','VV','tZq','fakeL','plj']
fdir='/home/pku/anying/cms/rootfiles/WWg/2018/'

In [3]:
f1 = uproot.open(fdir+'cutlep-out'+root_files[0]+'18.root')
tree=f1['Events']

In [5]:
necessary_cols=['lep1pt','lep1eta','lep1phi','lep2pt','lep2eta','lep2phi',
                'lep1_charge','lep2_charge','lep1_isprompt','lep2_isprompt',
                'lep1_is_tight','lep2_is_tight','n_loose_ele','n_loose_mu',
                'HLT_ee','HLT_mm','HLT_emu','ptll','mll','drll',
                'photonet','photoneta','photonphi','drl1a',
                'drl2a','mllg','PuppiMET_T1Smear_pt','PuppiMET_T1Smear_phi',
                'njets30','n_bjets20_medium','n_bjets20_loose',
                'scalef','ele_id_scale','ele_reco_scale','muon_id_scale','muon_iso_scale',
                'photon_id_scale','photon_veto_scale','btag_weight_medium'] 

In [ ]:
f = uproot.lazy(fdir+'cutlep-out'+root_files[0]+'18.root:Events')
events = ak.to_pandas(f[necessary_cols])

In [124]:
i=0
a=ROOT.TLorentzVector(events['lep1pt'].iloc[i],events['lep1eta'].iloc[i],
                                   events['lep1phi'].iloc[i],0.105666)


In [128]:
events['lep1pt'].iloc[i]

230.5769

In [129]:
a.Pt()

230.58875629072793

In [114]:
def get_df(root_file,year):
    f = uproot.lazy(fdir+'cutlep-out'+root_file+year+'.root:Events')
    events = ak.to_pandas(f[necessary_cols])
    if 'WWG_emu' in root_file:
        events.loc[events.index,'isSignal']=1
    else:
        events.loc[events.index,'isSignal']=1